In [ ]:
!pip -q install spacy ftfy unidecode wordcloud matplotlib pandas scikit-learn lxml
!python -m spacy download en_core_news_sm

     ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
     ---------- ----------------------------- 3.4/13.0 MB 21.2 MB/s eta 0:00:01
     ------------------------- -------------- 8.1/13.0 MB 21.4 MB/s eta 0:00:01
     ------------------------------------- - 12.6/13.0 MB 21.1 MB/s eta 0:00:01
     --------------------------------------- 13.0/13.0 MB 20.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


**Limpeza de texto:**

In [13]:
import re, html, json
from collections import Counter
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
from ftfy import fix_text
from sklearn.feature_extraction.text import TfidfVectorizer
#POR AQUILO QUE VI, NAO EXISTEM NENHUM DOMAIN TRASH A REMOVER, nao sei se entao metemos ou nao 


def basic_clean(text: str,
                lower=True,
                strip_accents=True,   
                keep_hyphen=True) -> str:  #preserva os hifens
    """Basic text cleaning for English hotel reviews."""
    if not isinstance(text, str):
        text = "" if text is None else str(text)
    t = text

    # corrigir problemas de encoding
    t = fix_text(t)

    # remove tags HTML(apesar de nao ter encontrado nenhuma)
    t = re.sub(r"<[^>]+>", " ", t)

    # remove URLs(apesar de nao ter encontrado nenhuma )
    t = re.sub(r"http\S+|www\.\S+", " ", t)

    #remove nao caracteres do dataset
    if keep_hyphen:
        t = re.sub(r"[^0-9A-Za-z\- ]", " ", t)
    else:
        t = re.sub(r"[^0-9A-Za-z ]", " ", t)

    # normalizar hífens e espaços
    t = re.sub(r"-{2,}", "-", t)
    t = re.sub(r"\s*-\s*", "-", t)
    t = re.sub(r"\s+", " ", t).strip()

    # lowercase
    if lower:
        t = t.lower()

    # remove acentos 
    if strip_accents:
        t = unidecode(t)

    return t


def pre_clean(s: str) -> str:
    """Corrigir unicode + limpar.""" # nao meti aqui para decodificar entidades HTML, uma vez que nao temos 
    if not isinstance(s, str):
        s = "" if s is None else str(s)
    t = fix_text(s)
    t = basic_clean(t, lower=True, strip_accents=False, keep_hyphen=True)
    return t





**Leitura do csv:**

In [14]:
df = pd.read_csv("Hotel_Reviews.csv")

# Negative reviews
df["clean_negative"] = df["Negative_Review"].apply(pre_clean)

# Positive reviews
df["clean_positive"] = df["Positive_Review"].apply(pre_clean)


**Comparação de colunas alteradas com as originais:**

In [15]:
# Ver só as linhas onde a review negativa mudou após limpeza
diff_neg = df[df["Negative_Review"] != df["clean_negative"]]
print(diff_neg[["Negative_Review", "clean_negative"]].head(10))  # mostra as primeiras 10 diferenças

# Para reviews positivas
diff_pos = df[df["Positive_Review"] != df["clean_positive"]]
print(diff_pos[["Positive_Review", "clean_positive"]].head(10))


                                     Negative_Review  \
0   I am so angry that i made this post available...   
1                                        No Negative   
2   Rooms are nice but for elderly a bit difficul...   
3   My room was dirty and I was afraid to walk ba...   
4   You When I booked with your company on line y...   
5   Backyard of the hotel is total mess shouldn t...   
6   Cleaner did not change our sheet and duvet ev...   
7   Apart from the price for the brekfast Everyth...   
8   Even though the pictures show very clean room...   
9   The aircondition makes so much noise and its ...   

                                      clean_negative  
0  i am so angry that i made this post available ...  
1                                        no negative  
2  rooms are nice but for elderly a bit difficult...  
3  my room was dirty and i was afraid to walk bar...  
4  you when i booked with your company on line yo...  
5  backyard of the hotel is total mess shouldn t ... 

In [16]:
import spacy

nlp = spacy.load("en_core_web_sm")
EN_STOP = nlp.Defaults.stop_words #default stop words em ingles

def spacy_tokenize_lemmatize(doc_text: str,
                             remove_stop=True,
                             only_alpha=True,
                             min_len=3) -> list[str]:
    """Tokeniza com spaCy e devolve lemas filtrados."""
    if not isinstance(doc_text, str) or not doc_text.strip():
        return []
    
    doc = nlp(doc_text)
    out = []
    for tok in doc:
        lemma = tok.lemma_ if tok.lemma_ != "" else tok.text
        if remove_stop and lemma.lower() in EN_STOP:
            continue
        if only_alpha and not lemma.isalpha():
            continue
        if len(lemma) < min_len:
            continue
        out.append(lemma.lower())
    return out

In [17]:
import pandas as pd

# Lê o ficheiro
df = pd.read_csv("Hotel_Reviews.csv")

# Combinar as colunas num único texto
df["review"] = df["Positive_Review"].fillna("") + " " + df["Negative_Review"].fillna("")

# (Opcional) remover espaços duplos
df["review"] = df["review"].str.strip()

# df = df.drop(columns=["positive_review", "negative_review"]) -> ver se fazemos drop ou nao


In [11]:
# Limpeza
df["clean_text"] = df["review"].apply(pre_clean)

# Tokenização + lematização
df["tokens"] = df["clean_text"].apply(lambda t: spacy_tokenize_lemmatize(
    t, remove_stop=True, only_alpha=True, min_len=3
))

print("Exemplo tokens do documento 0:", df["tokens"].iloc[0][:20])

# Guardar o resultado
df.to_csv("Hotel_Reviews_processed.csv", index=False)


Exemplo tokens do documento 0: ['park', 'outside', 'hotel', 'beautiful', 'angry', 'post', 'available', 'possible', 'site', 'use', 'plane', 'trip', 'mistake', 'book', 'place', 'booking', 'book', 'com', 'stay', 'night']


In [ ]:
import ollama
import pandas as pd
from tqdm import tqdm # Para veres a barra de progresso
tqdm.pandas()

print("A carregar dados...")
df = pd.read_csv("Hotel_Reviews_processed.csv")

# Se quiseres testar rápido, descomenta a linha abaixo para fazer só 100 linhas
df = df.head(100)

print("A gerar embeddings com Ollama (isto vai demorar)...")
# Função para aplicar a cada linha
def get_emb(text):
    try:
        # Certifica-te que tens o 'ollama serve' ligado no terminal!
        return ollama.embeddings(model='nomic-embed-text', prompt=text)['embedding']
    except:
        return []

# Cria a coluna nova com os números
df['embeddings'] = df['review'].progress_apply(get_emb)

# Guarda num ficheiro novo (PICKLE é melhor que CSV para guardar vetores/listas)
print("A guardar ficheiro pronto...")
df.to_pickle("dados_com_embeddings.pkl")
print("Concluído!")